In [2]:
%cd ..

/Users/mac/Monthy/Crawl_buffett


In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import re

In [4]:
driver = webdriver.Chrome()

In [5]:
url = 'https://www.proxynova.com/proxy-server-list/country-vn'
driver.get(url)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
tables = soup.find('table', id='tbl_proxy_list')
tbody = tables.find('tbody')

pattern = r'(\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})'
ip_address = re.findall(pattern, tbody.text)

df_proxy = pd.read_html(str(tables))[0].dropna(how = 'all')
df_proxy['Proxy IP'] = ip_address
driver.close()

In [6]:
df_proxy

,Proxy IP,Proxy Port,Last Check,Proxy Speed,Uptime,Proxy Country,Anonymity,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,113.161.59.136,8080.0,14 mins ago,3764 ms,10% (758),Vietnam - Ho Chi Minh City,Transparent,NaN,NaN,NaN
1,118.70.12.171,53281.0,14 mins ago,3395 ms,10% (398),Vietnam - Hanoi,Elite,NaN,NaN,NaN
2,103.155.217.156,41489.0,14 mins ago,1189 ms,42% (97),Vietnam,Elite,NaN,NaN,NaN
3,103.3.246.215,3128.0,15 mins ago,1803 ms,50% (866),Vietnam - Tinh Binh Duong,Transparent,NaN,NaN,NaN
4,14.161.26.10,8080.0,15 mins ago,3740 ms,11% (514),Vietnam - Ho Chi Minh City,Transparent,NaN,NaN,NaN
5,103.168.53.157,41495.0,15 mins ago,1017 ms,21% (97),Vietnam,Elite,NaN,NaN,NaN
6,113.160.209.113,19132.0,15 mins ago,2888 ms,8% (196),Vietnam - Hanoi,Transparent,NaN,NaN,NaN
7,113.160.214.209,19132.0,15 mins ago,3914 ms,13% (372),Vietnam - Hanoi,Transparent,NaN,NaN,NaN
8,103.74.121.88,3128.0,15 mins ago,2307 ms,51% (332),Vietnam,Transparent,NaN,NaN,NaN
9,103.168.53.105,41406.0,15 mins ago,977 ms,24% (99),Vietnam,Elite,NaN,NaN,NaN


In [7]:
import time
from selenium import webdriver
for i in df_proxy.index:
    proxy = df_proxy.loc[i, 'Proxy IP']
    port = int(df_proxy.loc[i, 'Proxy Port'])
    PROXY = f"{proxy}:{port}"
    print(PROXY, end = ' -- ')
    try:
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument('--proxy-server=%s' % PROXY)
        chrome = webdriver.Chrome(options=chrome_options)
        chrome.implicitly_wait(10)
        chrome.get('https://www.buffett-code.com/')
        soup = BeautifulSoup(
            chrome.page_source, "html.parser", from_encoding="utf-8"
        )
        if 'バフェット・コード' in chrome.page_source:
            print('success')
        else:
            print('error')
    except:
        print('error')
        continue
    chrome.quit()

113.161.59.136:8080 -- 

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/bs4/__init__.py:228: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


success
118.70.12.171:53281 -- success
103.155.217.156:41489 -- error
103.3.246.215:3128 -- success
14.161.26.10:8080 -- error
103.168.53.157:41495 -- error
113.160.209.113:19132 -- success
113.160.214.209:19132 -- error
103.74.121.88:3128 -- error
103.168.53.105:41406 -- error
116.102.111.5:8080 -- error
202.43.110.171:8080 -- success
103.141.143.1:41472 -- error
103.168.53.105:41447 -- error
103.168.53.105:41421 -- error
103.155.217.105:41405 -- error
27.77.146.21:10029 -- success
103.92.26.190:4002 -- success
103.141.143.1:41485 -- error
202.43.110.168:8080 -- error
103.141.142.153:41416 -- error
103.176.108.1:1338 -- error
113.160.154.23:8080 -- error
14.241.110.154:8080 -- success
103.155.217.156:41484 -- error
117.3.247.209:10014 -- success
103.155.217.156:41491 -- error
103.141.143.1:41487 -- error
103.168.53.105:41432 -- error
14.161.31.192:53281 -- error
103.168.53.157:41471 -- error
103.141.143.1:41491 -- error
103.168.53.157:41487 -- error
103.168.53.105:41418 -- error
103.1